# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

__1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.__

In [1]:
import pandas as pd

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
# max(np.array(list(map(len, obj.keys()))) + np.array(list((map(len, obj.values())))))
max(list(map(len, obj.keys())))

9

In [3]:
for key, value in obj.items():
    d = max(list(map(len, obj.keys())))
    print(f'{key:<{d}} = "{value}"')

home_page = "https://github.com/pypa/sampleproject"
keywords  = "sample setuptools development"
license   = "MIT"


__2. Написать регулярное выражение,которое позволит найти номера групп студентов.__

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

Примеры:
* **r (row)** - для вывода служебных выражений
* **\d (digital)** - цифра
* **\s (space)** - пробел
* **?** - 0/1
* _*_ - 0 или более
* **+** - 1 или более
* **(...)** - объединение символов в группу
* **(?:)** - non-capturing группа
* **\символ** - выведет "символ", *например, \+ выведет +*

In [5]:
import re

patt = re.compile(r"(?:ПМ)?\s?\d+-\d", re.I)

for item in obj:
    print(f'{item:20}', patt.findall(item))

Евгения гр.ПМ19-1    ['ПМ19-1']
Илья пм 20-4         ['пм 20-4']
Анна 20-3            [' 20-3']


__3. Разбейте текст формулировки задачи 2 на слова.__

text = 'Написать регулярное выражение,которое позволит найти номера групп студентов.'

ntlk.word_tokenize(text)

tokenizer = nltk.tokenize.RegexpTokenizer('r\w+')
tokenizer.tokenize(text)

## Лабораторная работа 6

In [6]:
import pandas as pd
import numpy as np

### Форматирование строк

**1\.** Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [7]:
recipes = pd.read_csv('recipes_sample.csv')
recipes.head(1)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0


In [8]:
five_recipes = recipes.iloc[np.random.choice(recipes.index.values, size=5, replace=False).tolist()][['id','minutes']].values
dict_5_recipes = {i: five_recipes[i].tolist() for i in range(len(five_recipes))}

width_id = max((list(map(lambda x: len(str(x)), five_recipes.T[0]))) + list([len('id')])) + 2
width_min = max((list(map(lambda x: len(str(x)), five_recipes.T[1]))) + list([len('minutes')])) + 2

print(f'|{"id":^{width_id}}|{"minutes":^{width_min}}|')
print(f'|{"-"*(width_id+width_min+1)}|')
for value in dict_5_recipes:
    print(f'|{dict_5_recipes[value][0]:^{width_id}}|{dict_5_recipes[value][1]:^{width_min}}|')

|   id   | minutes |
|------------------|
| 79414  |   40    |
| 102163 |   35    |
| 199548 |   30    |
| 177786 |   35    |
| 469496 |   45    |


**2\.** Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1.
2. Шаг 2.
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
with open('.\\steps_sample.xml') as f:
    stps = BeautifulSoup(f, 'xml')

Функция для вывода 

In [11]:
def show_info(name, steps, minutes, author_id):    
    final_string = f'"{name.title()}"\n\n'    
    for step, step_name in enumerate(steps,1):
        final_string += f'{step}. {step_name.capitalize()}\n'
    final_string += f'{"-" * 10}\n'
    final_string += f'Автор: {author_id}\n'
    final_string += f'Среднее время приготовления: {minutes} минут\n'
    return(final_string)    

In [12]:
recipe_id = 170895

steps_list = list()
for recipe in stps.recipes.find_all('recipe'):
    s = recipe.find("id").next
    if (int(s) == recipe_id):
        steps_list.append([step.next for step in recipe.find_all('step')])

recipes_info = {'Название': recipes[recipes.id==recipe_id]['name'].values[0],
                'Автор': recipes[recipes.id==recipe_id]['contributor_id'].values[0],
                'Среднее время приготовления': recipes[recipes.id==recipe_id]['minutes'].values[0]}

print(show_info(recipes_info['Название'], 
                steps_list[0],
                recipes_info["Среднее время приготовления"],
                recipes_info["Автор"]))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [13]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

**3\.** Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [14]:
recipe_id = 25082

steps_list = list()
for recipe in stps.recipes.find_all('recipe'):
    s = recipe.find("id").next
    if (int(s) == recipe_id):
        steps_list.append([step.next for step in recipe.find_all('step')])

In [15]:
for step, step_name in enumerate(steps_list[0],1):
    patt = re.compile(r"(\d+\s\bhours?\b|\d+\s\bminutes?\b)", re.I)
    if(len(patt.findall(step_name))!=0):
        print(f'{step}. {step_name}')
        print(f'\033[1m {patt.findall(step_name)} \033[0m \n')

6. turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
 ['20 minutes']  

8. when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
 ['10 minutes']  

10. let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours
 ['2 hours']  

14. when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc
 ['10 minutes']  

17. bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom
 ['20 minutes', '30 minutes']  



**4\.** Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [16]:
thisbut_df = recipes.iloc[np.where(recipes.description.str.extract(r"(^this(?:\s|\w)*?,\s*?but)").isnull()==False)[0]]
thisbut_df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
76,neezy peazy lunch,24760,35,29291,2002-04-08,NaN,"this is a great meal eaten the same day ,but e...",6.0
183,3 pepper quiche,311313,75,186802,2008-07-02,NaN,this was adapted from a recipe i found on the ...,13.0
337,addictive chex mix,277562,21,260464,2008-01-08,NaN,this is kind of similar to some of the other v...,12.0
486,almas pite hungarian apple cake,200227,1500,388615,2006-12-11,18.0,"this is a moist, buttery apple cake. the doug...",10.0
678,amish tears on your pillow pie,40858,75,6258,2002-09-20,NaN,this pie does not have very many ingredients i...,6.0
...,...,...,...,...,...,...,...,...
29315,white spinach lasagna,92419,85,101034,2004-06-01,12.0,this is so simple to put together and tastes s...,11.0
29340,whole rump barbecued,131322,1455,214855,2005-07-27,7.0,"this is so good, but does require 24 hours mar...",NaN
29853,zesty salad with tortilla strips,170416,9,169969,2006-05-30,10.0,this simple salad pairs wonderfully with any m...,11.0
29967,zucchini spice cake,80067,65,4439,2004-01-02,11.0,"this recipe is pretty close to bizzare, but it...",4.0


In [17]:
print(f'Всего рецептов с описанием вида "this..., but": \033[1m{len(thisbut_df)} шт\033[0m.\n\nПримеры описаний:')
sample = thisbut_df.description.sample(n=3)
sample_dict = dict(zip(sample.index.tolist(), sample.tolist()))

for id, description in sample_dict.items():
    print(f'\033[1m{id=}\033[0m: {description}\n')

Всего рецептов с описанием вида "this..., but": 134 шт.

Примеры описаний:
id=23704: this sauce is more sweet than sour, but still has a good "twang" all the same.  i used pineapple chunks because i had some leftover fresh pineapple in the freezer.  you can used canned, if that's what you have, but the taste just doesn't compare to the fresh!  if you prefer your sauce a bit less sweet, use canned pineapple and reduce the amount of brown sugar.  we just loved this recipe, so i thought some of you might, too.

id=22399: this is a recipe that i clipped from a magazine years ago, but unfortunately can't remember which one.  it was a makeover version of a person's favorite high fat casserole into a reduced fat/calorie dish.  my husband and i really enjoy this.  i usually serve it with a salad or steamed vegetable for a complete dinner.  (note: cooking time does not include time to cook brown rice.)

id=13505: this recipe is very similar to other feta chicken recipes on recipezaar, but i lik

**5\.** В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [18]:
recipe_id = 170895
patt = re.compile(r".*\d+\s\/\s\d+.*", re.I)    

In [19]:
steps_list = list()
for recipe in stps.recipes.find_all('recipe'):
    if (int(recipe.find("id").next) == recipe_id):
         for step in recipe.find_all("step"): 
                if(len(patt.findall(step.next))!=0):
                    steps_list.append(re.sub(r'\s\/\s', '/',step.next))
                else:
                    steps_list.append(step.next)

for step, step_name in enumerate(steps_list,1):
    print(f'{step}. {step_name}')

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an additional 1/2 teaspoon salt
21. serve warm


### Сегментация текста

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**6\.** Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [21]:
rec_glossary = list()

In [22]:
steps_list = list()
for recipe in stps.recipes.find_all('recipe'):
        for step in recipe.find_all("step"): 
            words = list(set(nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+').tokenize(step.next)))
#             for word in words:
#                 word.lower()
            rec_glossary.extend(words)
rec_glossary = list(set(rec_glossary))

In [23]:
rec_glossary[:5]

['believes', 'adequately', 'experimenting', 'online', 'sticking']

In [24]:
(f'Итого {len(rec_glossary)} уникальных слов(а) в рецепте')

'Итого 15139 уникальных слов(а) в рецепте'

**7\.** Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [25]:
recipes["description_length"] = recipes.description.apply(lambda x: len(nltk.sent_tokenize(str(x))))

In [26]:
recipes.sort_values('description_length',ascending=False).head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,description_length
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,24
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,23
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,23


**8\.** Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.

In [27]:
# nltk.download('averaged_perceptron_tagger')

In [28]:
recipe_id = 241106

steps_list = list()
for recipe in stps.recipes.find_all('recipe'):
    s = recipe.find("id").next
    if (int(s) == recipe_id):
        for step in recipe.find_all('step'):
            s = nltk.pos_tag(nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+').tokenize(step.next))
            dliny = list(map(len, list(list(zip(*s))[0])))
            for i, word in enumerate(list(list(zip(*s))[1])):
                if(len(word)<=dliny[i]):
                    print(f'{word:^{dliny[i]}}', end=' ')
                else:
                    print(f'{word:^{dliny[i]}}', end='')
            print(' ')
            print(*list(list(zip(*s))[0]), sep =' ')
            print(' ')

TO  VB  DT     JJ       NN      VBP   DT   JJ       NNS       RB    VBG   DT  NN  CC    VBG      RB    TO   VB     RB   
to make the balsamic marinade combine the first ingredients slowly adding the oil and stirring briskly to combine well
 
VBN  RP    
set aside
 
  VB    DTJJ     NN     NN  CC   VB    DTNN   NN   TO   VB    JJ   
prepare a hot charcoal fire or preheat a gas grill to medium high
 
 NN   DT   JJ        NN     NN  IN DT   NN    
place a vegetable grilling rack on the grill
 
CC   VB    DT    NN     
or preheat the broiler
 
TO  VB  DT     NN     VBZ   VBD DT     JJ        NN      IN   CD   NNS   TO    VB     NNS    
to make the eggplant steaks cut the eggplant lengthwise into four slices to resemble steaks
 
 IN   DT   NNS    IN  DT     NN     
brush the steaks with the marinade
 
 NN   CC  VB   DT     NN    IN    NNS   IN  DT   NN  CC  IN     NN   CC  RB   JJ   
grill or broil the eggplant for minutes on each side or until tender but not soft
 
  NN    IN  DT   NN  CC 